# **Explainability Evaluation Agent**

To evaluate the performance of our recommender system, we will analyze the system regarding the individual agents’ performance, the cold start problem, the sensitivity to changes in the recommendation hyperparameter and potential energy cost savings. We will introduce a further agent called Evaluation Agent which will perform all evaluation actions.

We will perform our evaluation analysis for households 1 to 10 in the REFIT: Electrical Load Measurements data (Murray et at., 2019), to validate our evaluation results.


## **1. Preparing the Environment**

In [1]:
from agents import Performance_Evaluation_Agent
from helper_functions import Helper

helper = Helper()

import pandas as pd
import numpy as np
import json
import pickle
from copy import deepcopy
import time
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
#import statsmodels.api as sm

import scipy.spatial

In [2]:
DATA_PATH = '../data/'
EXPORT_PATH = '../export/'

<br>

## **2. Preparations for Evaluating the Performance of our Recommender System**
### **2.1 Determining User Input**

Before we are able to use our recommender system and evaluate its performance, we need to specify the required user inputs (i.e. active appliances, shiftable devices and the consumption threshold). For specifying which of the devices in the respective household will be determined as active appliances and shiftable devices, we look at the description of the devices provided in the readme file and categorize the devices according to our definitions of the categories. Furthermore, we validate that the used devices do not contain any noise in their consumption data and remove devices which contain noise, i.e. consume energy constantly over time. 

In [3]:
readme = DATA_PATH+'REFIT_Readme.txt'
readme = helper.load_txt(readme)
start = readme.rfind('House 1\n')
end = readme.find('House 11\n')
print(readme[start:end])

House 1
0.Aggregate, 1.Fridge, 2.Chest Freezer, 3.Upright Freezer, 4.Tumble Dryer,
5.Washing Machine, 6.Dishwasher, 7.Computer Site, 8.Television Site, 9.Electric Heater
	!NOTES
		0. October 2014, numerous light bulbs changed to LEDs.
		7.a Desktop Computer
		7.b Computer Monitor

House 2
0.Aggregate, 1.Fridge-Freezer, 2.Washing Machine, 3.Dishwasher, 4.Television,
5.Microwave, 6.Toaster, 7.Hi-Fi, 8.Kettle, 9.Oven Extractor Fan
	!NOTES

House 3
0.Aggregate, 1.Toaster, 2.Fridge-Freezer, 3.Freezer, 4.Tumble Dryer,
5.Dishwasher, 6.Washing Machine, 7.Television, 8.Microwave, 9.Kettle
	!NOTES

House 4
0.Aggregate, 1.Fridge, 2.Freezer, 3.Fridge-Freezer, 4.Washing Machine (1),
5.Washing Machine (2), 6.Computer Site, 7.Television Site, 8.Microwave, 9.Kettle
	!NOTES
		6.a Desktop Computer
		6.b Computer Monitor
		6.c Scanner
		6.d Printer
		6.e Router
		6. Change in signature 01/02/2015
		7.a Television
		7.b DVD Player
		7.c VHS Player
		7. Change in signature 19/12/2014

House 5
0.Aggregate, 

<br>

**Shiftable Devices**

In [4]:
# manual input, determined using information provided in the readme
# validated in the next step using the validate thresholds functionality
shiftable_devices = {
    1: ['Tumble Dryer', 'Washing Machine', 'Dishwasher'],
    2: ['Washing Machine', 'Dishwasher'],
    3: ['Tumble Dryer', 'Washing Machine', 'Dishwasher'],
    4: ['Washing Machine (1)', 'Washing Machine (2)'],
    5: ['Tumble Dryer'], # , 'Washing Machine' --> consumes energy constantly; , 'Dishwasher' --> noise at 3am
    6: ['Washing Machine', 'Dishwasher'],
    7: ['Tumble Dryer', 'Washing Machine', 'Dishwasher'],
    8: ['Washing Machine'], # 'Dryer' --> consumes constantly
    9: ['Washer Dryer', 'Washing Machine', 'Dishwasher'], 
    10: ['Washing Machine'] #'Dishwasher'
}

<br>

**Active Appliances**

In [5]:
# manual input, determined using information provided in the readme
# validated in the next step using the validate thresholds functionality
active_appliances = {
    1: deepcopy(shiftable_devices[1]) + ['Television Site', 'Computer Site'],
    2: deepcopy(shiftable_devices[2]) + ['Television', 'Microwave', 'Toaster', 'Hi-Fi', 'Kettle'],
    3: deepcopy(shiftable_devices[3]) + ['Toaster', 'Television', 'Microwave', 'Kettle'],
    4: deepcopy(shiftable_devices[4]) + ['Television Site', 'Kettle'], #'Microwave', 'Computer Site' --> consume energy constantly 
    5: deepcopy(shiftable_devices[5]) + ['Television Site', 'Combination Microwave', 'Kettle', 'Toaster'], # 'Computer Site', --> consumes energy constantly
    6: deepcopy(shiftable_devices[6]) + ['MJY Computer', 'Kettle', 'Toaster'], #, 'PGM Computer', 'Television Site' 'Microwave' --> consume energy constantly 
    7: deepcopy(shiftable_devices[7]) + ['Television Site', 'Toaster', 'Kettle'],
    8: deepcopy(shiftable_devices[8]) + ['Toaster', 'Kettle'], # 'Television Site', 'Computer' --> consume energy constantly
    9: deepcopy(shiftable_devices[9]) + ['Microwave', 'Kettle'], #'Television Site', 'Hi-Fi' --> consume energy constantly
    10: deepcopy(shiftable_devices[10]) + ['Magimix (Blender)', 'Microwave'] # 'Television Site' --> consume energy constantly
}

<br>

**Energy Consumption Threshold**

Our Preparation Agent will require the energy consumption threshold, which will determine if a device was used in a given period. This threshold will allow to reduce the impact of noise in the data. We will determine the optimal thresholds for the households using the Preparation Agent’s validate thresholds method. To demonstrate how noise in the consumption data occurs, we call the validate thresholds method for household 1. The consumption data regarding the Television Site in household 1 seems to contain daily noise around 3 am. We will choose the optimal threshold, such that the noise is removed from the data.

Furthermore, we will create our initial evaluation configuration file for each household which will contain the specified user input.

In [6]:
# validating the thresholds for household 1 to show noise in the data
household_id = 3

In [7]:
# creating the config including the user input
config =  {'data': {'household': deepcopy(household_id)}}
config['user_input'] = {
    'shiftable_devices': deepcopy(shiftable_devices[config['data']['household']]),
    'active_appliances': deepcopy(active_appliances[config['data']['household']])
}

# initializing the evaluation agent
model_type = "logit"

In [8]:
evaluation = Performance_Evaluation_Agent(DATA_PATH, model_type, config, load_data=True)

In [9]:
preparation = evaluation.preparation

# Data-Preparation
df_th = preparation.truncate(preparation.input)
df_th = preparation.scale(df_th)
df_th = helper.aggregate(df_th, '60T')

# Graphical analysis of candidate thresholds
#thresholds = [0] + list(np.geomspace(.01, .4, 5))
#preparation.validate_thresholds(df_th, thresholds, config['user_input']['active_appliances'])

In [10]:
thresholds = {
    1: 0.15,
    2: 0.01,
    3: 0.01, 
    4: 0.01, 
    5: 0.025,
    6: 0.065, 
    7: 0.01, 
    8: 0.01, # washing machine over night
    9: 0.01, 
    10: 0.01
}

<br>

### **2.2 Running our Pipeline**

Before we are able to analyze the performance of our recommender system, we need to calculate all outputs of all our agents and all recommendations possible based on the available data. To conveniently compute these outputs and recommendations for the households, we added a pipeline method to the Evaluation Agent which allows to run every agent of our recommender system for every available date iteratively. We will demonstrate its functionality by creating the recommendations for household 3. 

Additionally, we will use a further method of the Evaluation Agent to receive the default configuration for evaluating our recommender system.

In [11]:
# creating the config including the user input
config =  {'data': {'household': deepcopy(household_id)}}
config['user_input'] = {
    'shiftable_devices': deepcopy(shiftable_devices[config['data']['household']]),
    'active_appliances': deepcopy(active_appliances[config['data']['household']]),
    'threshold': deepcopy(thresholds[config['data']['household']])
}

<br>

**Preparing the data**

In [12]:
# calling the evaluation agent
evaluation = Performance_Evaluation_Agent(DATA_PATH, model_type, config)
evaluation.get_default_config('preparation')
evaluation.config

{'data': {'household': 3},
 'user_input': {'shiftable_devices': ['Tumble Dryer',
   'Washing Machine',
   'Dishwasher'],
  'active_appliances': ['Tumble Dryer',
   'Washing Machine',
   'Dishwasher',
   'Toaster',
   'Television',
   'Microwave',
   'Kettle'],
  'threshold': 0.01},
 'preparation': {'activity': {'truncate': {'features': 'all',
    'factor': 1.5,
    'verbose': 0},
   'scale': {'features': 'all', 'kind': 'MinMax', 'verbose': 0},
   'aggregate': {'resample_param': '60T'},
   'activity': {'active_appliances': ['Tumble Dryer',
     'Washing Machine',
     'Dishwasher',
     'Toaster',
     'Television',
     'Microwave',
     'Kettle'],
    'threshold': 0.01},
   'time': {'features': ['hour', 'day_name']},
   'activity_lag': {'features': ['activity'], 'lags': [24, 48, 72]}},
  'usage': {'truncate': {'features': 'all', 'factor': 1.5, 'verbose': 0},
   'scale': {'features': 'all', 'kind': 'MinMax', 'verbose': 0},
   'activity': {'active_appliances': ['Tumble Dryer',
     'Was

In [13]:
evaluation.pipeline('preparation')


[evaluation agent] Finished preparing the data for the activity agent.
[evaluation agent] Finished preparing the data for the usage agent.
[evaluation agent] Finished preparing the data for the load agent.


<br>

**Creating all recommendations**


In [14]:
evaluation.get_default_config(['activity', 'usage', 'load'])
evaluation.config

{'data': {'household': 3,
  'first_date': '2013-09-26',
  'last_date': '2015-06-01',
  'start_dates': {'activity': '2013-09-29',
   'usage': '2013-09-28',
   'load': '2013-09-26',
   'usage_tumble_dryer': '2013-09-28',
   'usage_washing_machine': '2013-09-28',
   'usage_dishwasher': '2013-09-28',
   'combined': '2013-09-29'}},
 'user_input': {'shiftable_devices': ['Tumble Dryer',
   'Washing Machine',
   'Dishwasher'],
  'active_appliances': ['Tumble Dryer',
   'Washing Machine',
   'Dishwasher',
   'Toaster',
   'Television',
   'Microwave',
   'Kettle'],
  'threshold': 0.01},
 'preparation': {'activity': {'truncate': {'features': 'all',
    'factor': 1.5,
    'verbose': 0},
   'scale': {'features': 'all', 'kind': 'MinMax', 'verbose': 0},
   'aggregate': {'resample_param': '60T'},
   'activity': {'active_appliances': ['Tumble Dryer',
     'Washing Machine',
     'Dishwasher',
     'Toaster',
     'Television',
     'Microwave',
     'Kettle'],
    'threshold': 0.01},
   'time': {'feat

In [15]:
activity_predictions, model = evaluation._pipeline_activity_usage_load("activity")

agent:		activity
progress: 	611/611
time:		[00:57<00:00]

Time
2015-06-01 00:00:00    0.100977
2015-06-01 01:00:00    0.105693
2015-06-01 02:00:00    0.110602
2015-06-01 03:00:00    0.115709
2015-06-01 04:00:00    0.121020
2015-06-01 05:00:00    0.126540
2015-06-01 06:00:00    0.132274
2015-06-01 07:00:00    0.813402
2015-06-01 08:00:00    0.821005
2015-06-01 09:00:00    0.689085
2015-06-01 10:00:00    0.699884
2015-06-01 11:00:00    0.710466
2015-06-01 12:00:00    0.720824
2015-06-01 13:00:00    0.855411
2015-06-01 14:00:00    0.861594
2015-06-01 15:00:00    0.867553
2015-06-01 16:00:00    0.587394
2015-06-01 17:00:00    0.878820
2015-06-01 18:00:00    0.884137
2015-06-01 19:00:00    0.889251
2015-06-01 20:00:00    0.894165
2015-06-01 21:00:00    0.898887
2015-06-01 22:00:00    0.811163
2015-06-01 23:00:00    0.265878
Freq: 60T, dtype: float64


In [16]:
#this takes the prediction out of nesting and puts it into own row
activity_predictions_test = activity_predictions.explode('Prediction')
activity_predictions_test

,Agent,Date,Prediction
1,activity,2013-09-30,0.244145
1,activity,2013-09-30,0.320797
1,activity,2013-09-30,0.323635
1,activity,2013-09-30,0.25141
1,activity,2013-09-30,0.336607
...,...,...,...
610,activity,2015-06-01,0.889251
610,activity,2015-06-01,0.894165
610,activity,2015-06-01,0.898887
610,activity,2015-06-01,0.811163


In [17]:
# just to create time (other way?)
row = 1
hour = 0
hours = []
while row <=len(activity_predictions_test):
    hours.append(hour)
    hour += 1
    row += 1
    if hour == 24:
        hour = 0

print(hours)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,

In [18]:
config

{'data': {'household': 3,
  'first_date': '2013-09-26',
  'last_date': '2015-06-01',
  'start_dates': {'activity': '2013-09-29',
   'usage': '2013-09-28',
   'load': '2013-09-26',
   'usage_tumble_dryer': '2013-09-28',
   'usage_washing_machine': '2013-09-28',
   'usage_dishwasher': '2013-09-28',
   'combined': '2013-09-29'}},
 'user_input': {'shiftable_devices': ['Tumble Dryer',
   'Washing Machine',
   'Dishwasher'],
  'active_appliances': ['Tumble Dryer',
   'Washing Machine',
   'Dishwasher',
   'Toaster',
   'Television',
   'Microwave',
   'Kettle'],
  'threshold': 0.01},
 'preparation': {'activity': {'truncate': {'features': 'all',
    'factor': 1.5,
    'verbose': 0},
   'scale': {'features': 'all', 'kind': 'MinMax', 'verbose': 0},
   'aggregate': {'resample_param': '60T'},
   'activity': {'active_appliances': ['Tumble Dryer',
     'Washing Machine',
     'Dishwasher',
     'Toaster',
     'Television',
     'Microwave',
     'Kettle'],
    'threshold': 0.01},
   'time': {'feat

In [19]:
activity_predictions_test['time'] = hours
activity_predictions_test

,Agent,Date,Prediction,time
1,activity,2013-09-30,0.244145,0
1,activity,2013-09-30,0.320797,1
1,activity,2013-09-30,0.323635,2
1,activity,2013-09-30,0.25141,3
1,activity,2013-09-30,0.336607,4
...,...,...,...,...
610,activity,2015-06-01,0.889251,19
610,activity,2015-06-01,0.894165,20
610,activity,2015-06-01,0.898887,21
610,activity,2015-06-01,0.811163,22


In [20]:
activity_treshold = config['preparation']['activity']['activity']['threshold']
active_predictions = activity_predictions_test[activity_predictions_test.Prediction > activity_treshold]

Get data for prediction with Lime and Shap.

In [21]:
# load household data for Household 1
household = helper.load_household(DATA_PATH, 3)

In [22]:
threshold = 0.01
active_appliances = ['Tumble Dryer', 'Dishwasher', 'Washing Machine','Television', 'Microwave', 'Kettle']
shiftable_devices = ['Tumble Dryer', 'Washing Machine', 'Dishwasher']
#model_types = ['logit', 'knn', 'ada', 'random forest']

In [23]:
#activity params
truncation_params = {
    'features': 'all',
    'factor': 1.5,
    'verbose': 0
}

scale_params = {
    'features': 'all',
    'kind': 'MinMax',
    'verbose': 0
}

aggregate_params = {
    'resample_param': '60T'
}

activity_params = {
    'active_appliances': active_appliances,
    'threshold': threshold
}

time_params = {
    'features': ['hour', 'day_name']
}

activity_lag_params = {
    'features': ['activity'],
    'lags': [24, 48, 72]
}

activity_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'activity': activity_params,
    'time': time_params,
    'activity_lag': activity_lag_params
}

#load agent
device_params = {
    'threshold': threshold
}

load_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'shiftable_devices': shiftable_devices,
    'device': device_params
}

#usage agent

device = {
    'threshold' : threshold}

aggregate_params24_H = {
    'resample_param': '24H'
}

usage_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'activity': activity_params,
    'aggregate_hour': aggregate_params,
    'aggregate_day': aggregate_params24_H,
    'time': time_params,
    'activity_lag': activity_lag_params,
    'shiftable_devices' : shiftable_devices,
    'device': device
}

In [24]:
from agents import Activity_Agent, Usage_Agent, Load_Agent, Price_Agent, Preparation_Agent
#get out X_train etc.
# Load pickle data
#activity_df = pd.read_pickle('../data/processed_pickle/activity_df.pkl')
prep = Preparation_Agent(household)
activity_df = prep.pipeline_activity(household, activity_pipe_params)

In [25]:
activity = Activity_Agent(activity_df)

In [26]:
date = '2016-07-09'
X_train, y_train, X_test, y_test = activity.train_test_split(activity_df, date)
activity_df.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape

((14752, 11), (13883, 10), (13883,), (0, 10), (0,))

# Evaluation Lime, Shap

In [27]:
predictive_models = [model]
predictive_models
#y_test.head(20)#.iloc[2]

[LogisticRegression(random_state=0)]

In [74]:
#14752

activity_predictions_test['date time']= activity_predictions_test['Date'].astype(str) + " " + activity_predictions_test['time'].astype(str)
activity_predictions_test

,Agent,Date,Prediction,time,date time
1,activity,2013-09-30,0.244145,0,2013-09-30 0
1,activity,2013-09-30,0.320797,1,2013-09-30 1
1,activity,2013-09-30,0.323635,2,2013-09-30 2
1,activity,2013-09-30,0.25141,3,2013-09-30 3
1,activity,2013-09-30,0.336607,4,2013-09-30 4
...,...,...,...,...,...
610,activity,2015-06-01,0.889251,19,2015-06-01 19
610,activity,2015-06-01,0.894165,20,2015-06-01 20
610,activity,2015-06-01,0.898887,21,2015-06-01 21
610,activity,2015-06-01,0.811163,22,2015-06-01 22


In [29]:
activity_df.columns

Index(['activity', 'hour', 'activity_lag_24', 'activity_lag_48',
       'activity_lag_72', 'day_name_Monday', 'day_name_Saturday',
       'day_name_Sunday', 'day_name_Thursday', 'day_name_Tuesday',
       'day_name_Wednesday'],
      dtype='object')

In [68]:
test = activity_df.reset_index()
new_act_df = deepcopy(test)
new_act_df['Date'] = test.Time.apply(lambda x: str(x)[:10])

In [69]:
new_act_df

,Time,activity,hour,activity_lag_24,activity_lag_48,activity_lag_72,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday,Date
0,2013-09-25 19:00:00,1,19,NaN,NaN,NaN,0,0,0,0,0,1,2013-09-25
1,2013-09-25 20:00:00,1,20,NaN,NaN,NaN,0,0,0,0,0,1,2013-09-25
2,2013-09-25 21:00:00,1,21,NaN,NaN,NaN,0,0,0,0,0,1,2013-09-25
3,2013-09-25 22:00:00,0,22,NaN,NaN,NaN,0,0,0,0,0,1,2013-09-25
4,2013-09-25 23:00:00,0,23,NaN,NaN,NaN,0,0,0,0,0,1,2013-09-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14747,2015-06-02 06:00:00,0,6,0.0,0.0,0.0,0,0,0,0,1,0,2015-06-02
14748,2015-06-02 07:00:00,1,7,1.0,1.0,1.0,0,0,0,0,1,0,2015-06-02
14749,2015-06-02 08:00:00,1,8,1.0,1.0,1.0,0,0,0,0,1,0,2015-06-02
14750,2015-06-02 09:00:00,1,9,1.0,1.0,1.0,0,0,0,0,1,0,2015-06-02


In [64]:
new_act_df = new_act_df.merge(activity_predictions_test, on = "Date",how='inner')

In [65]:
new_act_df

,Time,activity,hour,activity_lag_24,activity_lag_48,activity_lag_72,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday,Date,Agent_x,Prediction_x,time_x,Agent_y,Prediction_y,time_y
0,2013-09-30 00:00:00,0,0,0.0,0.0,0.0,1,0,0,0,0,0,2013-09-30,activity,0.244145,0,activity,0.244145,0
1,2013-09-30 00:00:00,0,0,0.0,0.0,0.0,1,0,0,0,0,0,2013-09-30,activity,0.244145,0,activity,0.320797,1
2,2013-09-30 00:00:00,0,0,0.0,0.0,0.0,1,0,0,0,0,0,2013-09-30,activity,0.244145,0,activity,0.323635,2
3,2013-09-30 00:00:00,0,0,0.0,0.0,0.0,1,0,0,0,0,0,2013-09-30,activity,0.244145,0,activity,0.25141,3
4,2013-09-30 00:00:00,0,0,0.0,0.0,0.0,1,0,0,0,0,0,2013-09-30,activity,0.244145,0,activity,0.336607,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8432635,2015-06-01 23:00:00,0,23,0.0,0.0,0.0,1,0,0,0,0,0,2015-06-01,activity,0.265878,23,activity,0.889251,19
8432636,2015-06-01 23:00:00,0,23,0.0,0.0,0.0,1,0,0,0,0,0,2015-06-01,activity,0.265878,23,activity,0.894165,20
8432637,2015-06-01 23:00:00,0,23,0.0,0.0,0.0,1,0,0,0,0,0,2015-06-01,activity,0.265878,23,activity,0.898887,21
8432638,2015-06-01 23:00:00,0,23,0.0,0.0,0.0,1,0,0,0,0,0,2015-06-01,activity,0.265878,23,activity,0.811163,22


In [31]:
### LIME Function
from lime import lime_tabular
import statistics

#local = 9 #the instance we want to explain
n_iter = 1
i = 0

data = {'Explainability Model': [],
        'Predictive Model': [],
        'Classifier': [],
        'Run Duration': [],
        'MAEE': [],
        'MSEE': []}
exp_eval_df = pd.DataFrame(data)

for local in range(0, len(activity_predictions)):
    for pred_model in predictive_models:
        classifier = pred_model
        print(classifier)

        print(pred_model)
        print(str(pred_model))

        if "KNeighbors" in str(pred_model):
            predictive_model = "KNN"
        elif "Random" in str(pred_model):
            predictive_model = "Random Forest"
        elif "Ada" in str(pred_model):
            predictive_model = "AdaBoost"
        else:
            predictive_model = "Unknown model"

        print(predictive_model)

        # LIME
        explainability_model = 'LIME'
        start_time = time.time()
        #run once
        explainer = lime_tabular.LimeTabularExplainer(training_data = np.array(X_train),
                                                  mode = "regression",
                                                  feature_names = X_train.columns,
                                                  categorical_features = [0])

        exp = explainer.explain_instance(data_row = X_test.iloc[local], #changed to somewhere where activity= 1
                                    predict_fn = pred_model.predict)

        exp.show_in_notebook(show_table = True)
        end_time = time.time()
        difference_time = end_time - start_time

        #compute MSEE:
        y_expl_i = exp.local_pred
        #print(y_expl_i)
        #SEE = (y_pred_i - y_expl_i)**2 #squared prediction error for this computation (repetition necessary for MSEE)

        rep = 0

        exp_list_abs = []
        exp_list_squ = []
        for rep in range(n_iter): #number of iterations for computing the diffferent lime models

            exp = explainer.explain_instance(data_row = X_test.iloc[local], #changed to somewhere where activity= 1
                                    predict_fn = pred_model.predict)
            exp_list_abs.append(y_pred_i-exp.local_pred)
            exp_list_squ.append((y_pred_i-exp.local_pred)**2)

        exp_np_abs =np.array(exp_list_abs)
        exp_np_squ =np.array(exp_list_squ)
        #exp_list_abs.to_numpy()
        #exp_list_abs.flatten()
        #print(exp_list_abs.flatten())

        MAEE = statistics.mean(exp_np_abs.flatten())
        MSEE = statistics.mean(exp_np_squ.flatten())
        print(MAEE)

        #MSEE = mean(exp_list_squ[0])
        #print(MSEE)

        #exp.as_list()
        exp_eval_df.loc[i+1] = [explainability_model, predictive_model, classifier, difference_time, MAEE, MSEE]

        i=i+1


        # SHAP

        explainability_model = 'SHAP'
        start_time = time.time()

        explainer = shap.KernelExplainer(pred_model.predict_proba, X_train)
        shap_values = explainer.shap_values(X_test.iloc[local,:])
        display(shap.force_plot(explainer.expected_value[1], shap_values[1], X_test.iloc[local,:]))

        end_time = time.time()
        difference_time = end_time - start_time

        #compute MSEE:
        rep = 0

        shap_list_abs = []
        shap_list_squ = []

        for rep in range(n_iter):
            explainer = shap.KernelExplainer(pred_model.predict_proba, X_train)
            shap_values = explainer.shap_values(X_test.iloc[local,:])
            print(shap_values)
            # first array = contribution to class 0
            # second array = contribution to class 1
            contribution_to_class_1 = np.array(shap_values).sum(axis=1)[1] # the red part of the diagram
            print(contribution_to_class_1)
            base_value = explainer.expected_value[1] # the mean prediction
            print(base_value)
            y_expl_i = base_value + contribution_to_class_1
            print(y_expl_i)
            SEE = (y_pred_i[0] - y_expl_i)**2 #squared prediction error for this computation (repetition necessary for MSEE)

            shap_list_abs.append(y_pred_i-y_expl_i)
            shap_list_squ.append((y_pred_i-y_expl_i)**2)

            print(shap_list_abs)

        shap_np_abs =np.array(shap_list_abs)
        shap_np_squ =np.array(shap_list_squ)

        MAEE = statistics.mean(shap_np_abs.flatten())
        MSEE = statistics.mean(shap_np_squ.flatten())
        print(MAEE)

        exp_eval_df.loc[i+1] = [explainability_model, predictive_model, classifier, difference_time, MAEE, MSEE]

        i = i+1

        print(exp_eval_df)

LogisticRegression(random_state=0)
LogisticRegression(random_state=0)
LogisticRegression(random_state=0)
Unknown model


IndexError: single positional indexer is out-of-bounds

In [ ]:
exp_eval_df